In [7]:
# import the usual
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns; sns.set()
from ipywidgets import interact, fixed
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

In [9]:
df_attendance = pd.read_csv('attendance.csv')
df_games = pd.read_csv('games.csv')
df_standings = pd.read_csv('standings.csv')

ValueError: could not convert string to float: 'Arizona'